In [151]:
# pip install mayavi

In [152]:
# import plotly.graph_objects as go
# import open3d as o3d
# import numpy as np

# def plot_ply_with_coordinates(ply_path):
#     # Read PLY file using Open3D
#     point_cloud = o3d.io.read_point_cloud(ply_path)
    
#     # Extract coordinates from the point cloud
#     coordinates = np.asarray(point_cloud.points)

#     # Create a 3D scatter plot
#     fig = go.Figure()

#     # Add the point cloud to the plot
#     fig.add_trace(go.Scatter3d(
#         x=coordinates[:, 0],
#         y=coordinates[:, 1],
#         z=coordinates[:, 2],
#         mode='markers',
#         marker=dict(
#             size=3,
#             color='blue',
#         ),
#         text=[str(i) for i in range(len(coordinates))],  # Use text for annotations
#         name='Point Cloud'
#     ))

#     # Set layout properties
#     fig.update_layout(
#         scene=dict(
#             aspectmode='data',
#             xaxis=dict(title='X'),
#             yaxis=dict(title='Y'),
#             zaxis=dict(title='Z'),
#         ),
#         title='PLY Visualization with Coordinates'
#     )

#     # Show the plot
#     fig.show()

# # Replace 'path/to/your/file.ply' with the actual path to your PLY file
# ply_file_path = './Preprocessed/id00012/mesh.ply'
# plot_ply_with_coordinates(ply_file_path)


In [153]:
from dataset_tf import VoxDataset
#import torch
#from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
import pandas as pd

In [154]:
dataset = VoxDataset()

In [155]:
mesh = dataset[0][1].numpy()
mesh

2024-04-21 02:21:29.404821: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-04-21 02:21:29.404851: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-21 02:21:29.404858: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-21 02:21:29.405099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-21 02:21:29.405448: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


array([ 0.32426515, -0.05228692, -0.2551459 , ..., -0.16584978,
        0.10881446,  0.04802914], dtype=float32)

In [156]:
# max_0 = 0
# min_0 = 0
# max_1 = 0
# min_1 = 0
# max_2 = 0
# min_2 = 0

# for i in tqdm(mesh):
#     if i[0] > max_0:
#         max_0 = i[0]
#     if i[1] > max_1:
#         max_1 = i[1]
#     if i[2] > max_2:
#         max_2 = i[2]

#     if i[0] < min_0:
#         min_0 = i[0]
#     if i[1] < min_1:
#         min_1 = i[1]
#     if i[2] < min_2:
#         min_2 = i[2]

In [157]:
# range_0 = max_0 - min_0
# range_0

In [158]:
# range_1 = max_1 - min_1
# range_1

In [159]:
# range_2 = max_2 - min_2
# range_2

In [1]:
import open3d as o3d

# Load the point cloud from a file (replace with your file format and path)
pcd = o3d.io.read_point_cloud("./Preprocessed/id00012/mesh.ply")  # Adjust for file format (e.g., las, xyz)

pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)


In [161]:
np.array(pcd.normals)

array([], shape=(0, 3), dtype=float64)

In [162]:
print("Paint the 1500th point red.")
pcd.colors[3526] = [1, 0, 0]

Paint the 1500th point red.


In [163]:
print("Find its 200 nearest neighbors, and paint them blue.")
# nose
[k, idx_1, _] = pcd_tree.search_knn_vector_3d(pcd.points[3526], 3500)

# trained on 3000

Find its 200 nearest neighbors, and paint them blue.


In [164]:
# chin
[k, idx_2, _] = pcd_tree.search_knn_vector_3d(pcd.points[3655], 3000)

In [165]:
[k, idx_3, _] = pcd_tree.search_knn_vector_3d(pcd.points[4051], 3000)

In [166]:
[k, idx_4, _] = pcd_tree.search_knn_vector_3d(pcd.points[4597], 3000)

In [167]:
# neck
[k, idx_5, _] = pcd_tree.search_knn_vector_3d(pcd.points[3285], 500)

In [168]:
# chin
[k, idx_6, _] = pcd_tree.search_knn_vector_3d(pcd.points[3404], 3000)

In [169]:
idx = np.array(idx_1).tolist() + np.array(idx_2).tolist() 
# + np.array(idx_3).tolist() + np.array(idx_4).tolist() + np.array(idx_5).tolist() + np.array(idx_6).tolist()

In [170]:
idx = list(set(idx))

In [171]:
len(idx)

3540

In [172]:
np.save("weight_indices.npy", np.array(idx))


In [173]:
np.asarray(pcd.colors)[idx[1:], :] = [1, 0, 0]

In [174]:
print("Find its neighbors with distance less than 0.2, and paint them green.")
[k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[3526], 0.2)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]

Find its neighbors with distance less than 0.2, and paint them green.


In [175]:
print("Visualize the point cloud.")
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.5599,
                                  front=[-0.4958, 0.8229, 0.2773],
                                  lookat=[2.1126, 1.0163, -1.8543],
                                  up=[0.1007, -0.2626, 0.9596], point_show_normal=True)

Visualize the point cloud.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [176]:
import numpy as np

# Create a NumPy matrix (or array) with dimensions (5023, 3) filled with zeros
zeros_matrix = np.ones((5023, 3))

# Print the matrix (optional)
print(zeros_matrix)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 ...
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


In [177]:
ids = [1, 2, 3]

In [178]:
for i in ids:
    zeros_matrix[i] = zeros_matrix[i] * 10

In [179]:
zeros_matrix

array([[ 1.,  1.,  1.],
       [10., 10., 10.],
       [10., 10., 10.],
       ...,
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.]])

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("output.csv")

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,audio_dir,emb_dir,mesh_dir,gender
0,0,0,id00012,./Preprocessed/id00012/00002.mp3,./Preprocessed/id00012/Embeddings/00002.mp3.npy,./Preprocessed/id00012/mesh.ply,m
1,1,1,id00012,./Preprocessed/id00012/00032.mp3,./Preprocessed/id00012/Embeddings/00032.mp3.npy,./Preprocessed/id00012/mesh.ply,m
2,2,2,id00012,./Preprocessed/id00012/00034.mp3,./Preprocessed/id00012/Embeddings/00034.mp3.npy,./Preprocessed/id00012/mesh.ply,m
3,3,3,id00012,./Preprocessed/id00012/00035.mp3,./Preprocessed/id00012/Embeddings/00035.mp3.npy,./Preprocessed/id00012/mesh.ply,m
4,4,4,id00012,./Preprocessed/id00012/00037.mp3,./Preprocessed/id00012/Embeddings/00037.mp3.npy,./Preprocessed/id00012/mesh.ply,m


In [4]:
value_counts = df['id'].value_counts()

In [9]:
[x for x in value_counts.tolist() if x != 20 ]

[]

In [10]:
value_counts

id
id00012    20
id05969    20
id05983    20
id05982    20
id05981    20
           ..
id02936    20
id02935    20
id02928    20
id02926    20
id08892    20
Name: count, Length: 5520, dtype: int64

In [6]:
df_1 = df[df['id'].isin(value_counts[value_counts >= 20].index)]

In [7]:
df_1['id'].value_counts()

id
id00012    20
id05969    20
id05983    20
id05982    20
id05981    20
           ..
id02936    20
id02935    20
id02928    20
id02926    20
id08892    20
Name: count, Length: 5520, dtype: int64

In [187]:
df_1.head()

,Unnamed: 0,id,audio_dir,emb_dir,mesh_dir,gender
0,0,id00012,./Preprocessed/id00012/00002.mp3,./Preprocessed/id00012/Embeddings/00002.mp3.npy,./Preprocessed/id00012/mesh.ply,m
1,1,id00012,./Preprocessed/id00012/00032.mp3,./Preprocessed/id00012/Embeddings/00032.mp3.npy,./Preprocessed/id00012/mesh.ply,m
2,2,id00012,./Preprocessed/id00012/00034.mp3,./Preprocessed/id00012/Embeddings/00034.mp3.npy,./Preprocessed/id00012/mesh.ply,m
3,3,id00012,./Preprocessed/id00012/00035.mp3,./Preprocessed/id00012/Embeddings/00035.mp3.npy,./Preprocessed/id00012/mesh.ply,m
4,4,id00012,./Preprocessed/id00012/00037.mp3,./Preprocessed/id00012/Embeddings/00037.mp3.npy,./Preprocessed/id00012/mesh.ply,m


In [188]:
df_1.to_csv("output.csv")

In [189]:
import pandas as pd


In [190]:
df1 = pd.read_csv('output.csv')

In [191]:
df1.head()

,Unnamed: 0.1,Unnamed: 0,id,audio_dir,emb_dir,mesh_dir,gender
0,0,0,id00012,./Preprocessed/id00012/00002.mp3,./Preprocessed/id00012/Embeddings/00002.mp3.npy,./Preprocessed/id00012/mesh.ply,m
1,1,1,id00012,./Preprocessed/id00012/00032.mp3,./Preprocessed/id00012/Embeddings/00032.mp3.npy,./Preprocessed/id00012/mesh.ply,m
2,2,2,id00012,./Preprocessed/id00012/00034.mp3,./Preprocessed/id00012/Embeddings/00034.mp3.npy,./Preprocessed/id00012/mesh.ply,m
3,3,3,id00012,./Preprocessed/id00012/00035.mp3,./Preprocessed/id00012/Embeddings/00035.mp3.npy,./Preprocessed/id00012/mesh.ply,m
4,4,4,id00012,./Preprocessed/id00012/00037.mp3,./Preprocessed/id00012/Embeddings/00037.mp3.npy,./Preprocessed/id00012/mesh.ply,m


In [192]:
df1.shape

(110400, 7)

In [193]:
df2 = pd.read_csv("vox2_meta.csv")

In [194]:
df2.head()

,id,VGGFace2 ID,Gender,Set
0,id00012,n000012,m,dev
1,id00015,n000015,m,dev
2,id00016,n000016,m,dev
3,id00017,n000017,m,test
4,id00018,n000018,m,dev


In [195]:
df2['id'] = df2['id'].apply(lambda x : x.strip())

In [196]:
df2['id'][0]

'id00012'

In [197]:
def get_gender(id):
    if id in df2['id'].tolist():
        return df2.loc[df2['id'] == id, 'Gender '].iloc[0].strip()

In [198]:
print(get_gender("id08887"))

m


In [199]:
df1['gender'] = df1['id'].apply(get_gender)

In [ ]:
df1.shape

(107300, 8)

In [ ]:
df1.to_csv('output.csv')

In [ ]:
df1[df1['gender'] == "m"]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,audio_dir,emb_dir,mesh_dir,gender
0,0,0,0,id00012,./Preprocessed/id00012/00002.mp3,./Preprocessed/id00012/Embeddings/00002.mp3.npy,./Preprocessed/id00012/mesh.ply,m
1,1,1,1,id00012,./Preprocessed/id00012/00032.mp3,./Preprocessed/id00012/Embeddings/00032.mp3.npy,./Preprocessed/id00012/mesh.ply,m
2,2,2,2,id00012,./Preprocessed/id00012/00034.mp3,./Preprocessed/id00012/Embeddings/00034.mp3.npy,./Preprocessed/id00012/mesh.ply,m
3,3,3,3,id00012,./Preprocessed/id00012/00035.mp3,./Preprocessed/id00012/Embeddings/00035.mp3.npy,./Preprocessed/id00012/mesh.ply,m
4,4,4,4,id00012,./Preprocessed/id00012/00037.mp3,./Preprocessed/id00012/Embeddings/00037.mp3.npy,./Preprocessed/id00012/mesh.ply,m
...,...,...,...,...,...,...,...,...
107295,107295,107576,107635,id08892,./Preprocessed/id08892/00053.mp3,./Preprocessed/id08892/Embeddings/00053.mp3.npy,./Preprocessed/id08892/mesh.ply,m
107296,107296,107577,107636,id08892,./Preprocessed/id08892/00054.mp3,./Preprocessed/id08892/Embeddings/00054.mp3.npy,./Preprocessed/id08892/mesh.ply,m
107297,107297,107578,107637,id08892,./Preprocessed/id08892/00055.mp3,./Preprocessed/id08892/Embeddings/00055.mp3.npy,./Preprocessed/id08892/mesh.ply,m
107298,107298,107579,107638,id08892,./Preprocessed/id08892/00057.mp3,./Preprocessed/id08892/Embeddings/00057.mp3.npy,./Preprocessed/id08892/mesh.ply,m


In [11]:
import numpy as np

In [30]:
ones = np.ones((5023,3))

In [31]:
ones

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [32]:
for index1, i in enumerate(ones):
    for index2, j in enumerate(i):
        ones[index1][index2] = (index1 * 3) + index2

In [33]:
ones

array([[0.0000e+00, 1.0000e+00, 2.0000e+00],
       [3.0000e+00, 4.0000e+00, 5.0000e+00],
       [6.0000e+00, 7.0000e+00, 8.0000e+00],
       ...,
       [1.5060e+04, 1.5061e+04, 1.5062e+04],
       [1.5063e+04, 1.5064e+04, 1.5065e+04],
       [1.5066e+04, 1.5067e+04, 1.5068e+04]])

In [34]:
ones[3516]

array([10548., 10549., 10550.])

In [ ]:
import numpy as np

zeros = np.zeros([1,15069])

In [ ]:
[nose_bridge,r_eye_lid,l_eye_lid,lip,r_lip_bend,l_lip_bend,r_lip_jaw,l_lip_jaw,lip_chin,orbital_lower,oribtal_upper,puffer,mouth_corner,jaw_end,ear_end]

In [ ]:
nose_bridge =  tf.math.reduce_euclidean_norm(y_test[3516],y_test[3526]) - tf.math.reduce_euclidean_norm(y_pred[3516],y_pred[3526])
r_eye_lid =  tf.math.reduce_euclidean_norm(y_test[3690], y_test[2265]) - tf.math.reduce_euclidean_norm(y_pred[3690], y_pred[2265])
l_eye_lid =  tf.math.reduce_euclidean_norm(y_test[3856], y_test[809]) - tf.math.reduce_euclidean_norm(y_pred[3856], y_pred[809])
lip =  tf.math.reduce_euclidean_norm(y_test[3543], y_test[3503]) - tf.math.reduce_euclidean_norm(y_pred[3543], y_pred[3503])
r_lip_bend =  tf.math.reduce_euclidean_norm(y_test[2850], y_test[3798]) - tf.math.reduce_euclidean_norm(y_pred[2850], y_pred[3798])
l_lip_bend =  tf.math.reduce_euclidean_norm(y_test[1735], y_test[3021]) - tf.math.reduce_euclidean_norm(y_pred[1735], y_pred[3021])
r_lip_jaw = tf.math.reduce_euclidean_norm(y_test[3798], y_test[3406]) - tf.math.reduce_euclidean_norm(y_pred[3798], y_pred[3406])
l_lip_jaw = tf.math.reduce_euclidean_norm(y_test[3021], y_test[3614]) - tf.math.reduce_euclidean_norm(y_pred[3021], y_pred[3614])
lip_chin =  tf.math.reduce_euclidean_norm(y_test[3503], y_test[3487]) - tf.math.reduce_euclidean_norm(y_pred[3503], y_pred[3487])
orbital_lower =  tf.math.reduce_euclidean_norm(y_test[3710], y_test[3866]) - tf.math.reduce_euclidean_norm(y_pred[3710], y_pred[3866])
oribtal_upper = tf.math.reduce_euclidean_norm(y_test[3154], y_test[2135]) - tf.math.reduce_euclidean_norm(y_pred[3154], y_pred[2135])
puffer = tf.math.reduce_euclidean_norm(y_test[3436], y_test[3667]) - tf.math.reduce_euclidean_norm(y_pred[3436], y_pred[3667])
mouth_corner = tf.math.reduce_euclidean_norm(y_test[2827], y_test[1710]) - tf.math.reduce_euclidean_norm(y_pred[2827], y_pred[1710])
jaw_end = tf.math.reduce_euclidean_norm(y_test[3406], y_test[3614]) - tf.math.reduce_euclidean_norm(y_pred[3406], y_pred[3614])
ear_end = tf.math.reduce_euclidean_norm(y_test[856], y_test[288]) - tf.math.reduce_euclidean_norm(y_pred[856], y_pred[288])

In [2]:
import numpy as np
from scipy.interpolate import LinearSpline, BSpline

def linear_interpolation(points):
  """
  Interpolates a straight line through 3D points.

  Args:
      points: A NumPy array of shape (n, 3) representing the 3D points.

  Returns:
      A function that takes a single parameter t (0 <= t <= 1)
      and returns the corresponding point on the line.
  """
  # Calculate starting and ending points
  start_point = points[0]
  end_point = points[-1]

  # Define direction vector
  direction = end_point - start_point

  def line_func(t):
    # Linear combination for interpolation
    return start_point + t * direction

  return line_func

def cubic_spline_interpolation(points):
  """
  Interpolates a smooth curve using a cubic B-spline through 3D points.

  Args:
      points: A NumPy array of shape (n, 3) representing the 3D points.

  Returns:
      A BSpline object representing the cubic spline curve.
  """
  # Convert points to NumPy array if necessary
  points = np.array(points)

  # Create a cubic B-spline interpolation
  spline = BSpline(np.arange(len(points)), points, degree=3)

  return spline

ImportError: cannot import name 'LinearSpline' from 'scipy.interpolate' (/Users/naweenk/miniforge3/envs/demo/lib/python3.10/site-packages/scipy/interpolate/__init__.py)

In [ ]:
line_func = linear_interpolation(pcd)
  # Example usage: point_on_line = line_func(0.5)  # Get point at t=0.5

  # Cubic spline interpolation
spline = cubic_spline_interpolation(pcd)

In [ ]:
# nose_bridge =  tf.norm(y_test[j,3516] - y_test[j,3526],ord='euclidean') - tf.norm(y_pred[j,3516] - y_pred[j,3526],ord='euclidean')
            # r_eye_lid =  tf.norm(y_test[j,3690] - y_test[j,2265],ord='euclidean') - tf.norm(y_pred[j,3690] - y_pred[j,2265],ord='euclidean')
            # l_eye_lid =  tf.norm(y_test[j,3856] - y_test[j,809],ord='euclidean') - tf.norm(y_pred[j,3856] - y_pred[j,809],ord='euclidean')
            # lip =  tf.norm(y_test[j,3543] - y_test[j,3503],ord='euclidean') - tf.norm(y_pred[j,3543] - y_pred[j,3503],ord='euclidean')
            # r_lip_bend =  tf.norm(y_test[j,2850] - y_test[j,3798],ord='euclidean') - tf.norm(y_pred[j,2850] - y_pred[j,3798],ord='euclidean')
            # l_lip_bend =  tf.norm(y_test[j,1735] - y_test[j,3021],ord='euclidean') - tf.norm(y_pred[j,1735] - y_pred[j,3021],ord='euclidean')
            # r_lip_jaw = tf.norm(y_test[j,3798] - y_test[j,3406],ord='euclidean') - tf.norm(y_pred[j,3798] - y_pred[j,3406],ord='euclidean')
            # l_lip_jaw = tf.norm(y_test[j,3021] - y_test[j,3614],ord='euclidean') - tf.norm(y_pred[j,3021] - y_pred[j,3614],ord='euclidean')
            # lip_chin =  tf.norm(y_test[j,3503] - y_test[j,3487],ord='euclidean') - tf.norm(y_pred[j,3503] - y_pred[j,3487],ord='euclidean')
            # orbital_lower =  tf.norm(y_test[j,3710] - y_test[j,3866],ord='euclidean') - tf.norm(y_pred[j,3710] - y_pred[j,3866],ord='euclidean')
            # oribtal_upper = tf.norm(y_test[j,3154] - y_test[j,2135],ord='euclidean') - tf.norm(y_pred[j,3154] - y_pred[j,2135],ord='euclidean')
            # puffer = tf.norm(y_test[j,3436] - y_test[j,3667],ord='euclidean') - tf.norm(y_pred[j,3436] - y_pred[j,3667],ord='euclidean')
            # mouth_corner = tf.norm(y_test[j,2827] - y_test[j,1710],ord='euclidean') - tf.norm(y_pred[j,2827] - y_pred[j,1710],ord='euclidean')
            # jaw_end = tf.norm(y_test[j,3406] - y_test[j,3614],ord='euclidean') - tf.norm(y_pred[j,3406] - y_pred[j,3614],ord='euclidean')
            # ear_end = tf.norm(y_test[j,856] - y_test[j,288],ord='euclidean') - tf.norm(y_pred[j,856] - y_pred[j,288],ord='euclidean')

            # diff_tot = [nose_bridge,r_eye_lid,l_eye_lid,lip,r_lip_bend,l_lip_bend,r_lip_jaw,l_lip_jaw,lip_chin,orbital_lower,oribtal_upper,puffer,mouth_corner,jaw_end,ear_end]


            # nose_bridge_l_loss = tf.norm(y_test[j,3516] - y_pred[j,3516],ord='euclidean')
            # r_eye_lid_l_loss = tf.norm(y_test[j,3690] - y_pred[j,3690],ord='euclidean')
            # l_eye_lid_l_loss = tf.norm(y_test[j,3856] - y_pred[j,3856],ord='euclidean')
            # lip_l_loss = tf.norm(y_test[j,3543] - y_pred[j,3543],ord='euclidean')
            # r_lip_bend_l_loss = tf.norm(y_test[j,2850] - y_pred[j,2850],ord='euclidean')
            # l_lip_bend_l_loss = tf.norm(y_test[j,1735] - y_pred[j,1735],ord='euclidean')
            # r_lip_jaw_l_loss = tf.norm(y_test[j,3798] - y_pred[j,3798],ord='euclidean')
            # l_lip_jaw_l_loss = tf.norm(y_test[j,3021] - y_pred[j,3021],ord='euclidean')
            # lip_chin_l_loss = tf.norm(y_test[j,3503] - y_pred[j,3503],ord='euclidean')
            # orbital_lower_l_loss = tf.norm(y_test[j,3710] - y_pred[j,3710],ord='euclidean')
            # oribtal_upper_l_loss = tf.norm(y_test[j,3154] - y_pred[j,3154],ord='euclidean')
            # puffer_l_loss = tf.norm(y_test[j,3436] - y_pred[j,3436],ord='euclidean')
            # mouth_corner_l_loss = tf.norm(y_test[j,2827] - y_pred[j,2827],ord='euclidean')
            # jaw_end_l_loss = tf.norm(y_test[j,3406] - y_pred[j,3406],ord='euclidean')
            # ear_end_l_loss = tf.norm(y_test[j,856] - y_pred[j,856],ord='euclidean')


            # nose_bridge_r_loss = tf.norm(y_test[j,3526] - y_pred[j,3526],ord='euclidean')
            # r_eye_lid_r_loss = tf.norm(y_test[j,2265] - y_pred[j,2265],ord='euclidean')
            # l_eye_lid_r_loss = tf.norm(y_test[j,809] - y_pred[j,809],ord='euclidean')
            # lip_r_loss = tf.norm(y_test[j,3503] - y_pred[j,3503],ord='euclidean')
            # r_lip_bend_r_loss = tf.norm(y_test[j,3798] - y_pred[j,3798],ord='euclidean')
            # l_lip_bend_r_loss = tf.norm(y_test[j,3021] - y_pred[j,3021],ord='euclidean')
            # r_lip_jaw_r_loss = tf.norm(y_test[j,3406] - y_pred[j,3406],ord='euclidean')
            # l_lip_jaw_r_loss = tf.norm(y_test[j,3614] - y_pred[j,3614],ord='euclidean')
            # lip_chin_r_loss = tf.norm(y_test[j,3487] - y_pred[j,3487],ord='euclidean')
            # orbital_lower_r_loss = tf.norm(y_test[j,3866] - y_pred[j,3866],ord='euclidean')
            # oribtal_upper_r_loss = tf.norm(y_test[j,2135] - y_pred[j,2135],ord='euclidean')
            # puffer_r_loss = tf.norm(y_test[j,3667] - y_pred[j,3667],ord='euclidean')
            # mouth_corner_r_loss = tf.norm(y_test[j,1710] - y_pred[j,1710],ord='euclidean')
            # jaw_end_r_loss = tf.norm(y_test[j,3614] - y_pred[j,3614],ord='euclidean')
            # ear_end_r_loss = tf.norm(y_test[j,288] - y_pred[j,288],ord='euclidean')

            # nose_bridge_loss = tf.norm(nose_bridge_l_loss - nose_bridge_r_loss, ord='euclidean') ** 2
            # r_eye_lid_loss = tf.norm(r_eye_lid_l_loss - r_eye_lid_r_loss , ord='euclidean') ** 2
            # l_eye_lid_loss = tf.norm(l_eye_lid_l_loss - l_eye_lid_r_loss, ord='euclidean') ** 2
            # lip_loss = tf.norm(lip_l_loss - lip_r_loss, ord='euclidean') ** 2
            # r_lip_bend_loss = tf.norm(r_lip_bend_l_loss - r_lip_bend_r_loss , ord='euclidean') ** 2
            # l_lip_bend_loss = tf.norm(l_lip_bend_l_loss - l_eye_lid_r_loss , ord='euclidean') ** 2
            # r_lip_jaw_loss = tf.norm(r_lip_jaw_l_loss - r_lip_jaw_r_loss , ord='euclidean') ** 2
            # l_lip_jaw_loss = tf.norm(l_lip_jaw_l_loss - l_lip_jaw_r_loss, ord='euclidean') ** 2
            # lip_chin_loss = tf.norm(lip_chin_l_loss - lip_chin_r_loss, ord='euclidean') ** 2
            # orbital_lower_loss = tf.norm(orbital_lower_l_loss - orbital_lower_r_loss , ord='euclidean') ** 2
            # oribtal_upper_loss = tf.norm(oribtal_upper_l_loss - oribtal_upper_r_loss, ord='euclidean') ** 2
            # puffer_loss = tf.norm(puffer_l_loss - puffer_r_loss, ord='euclidean') ** 2
            # mouth_corner_loss = tf.norm(mouth_corner_l_loss - mouth_corner_r_loss, ord='euclidean') ** 2
            # jaw_end_loss = tf.norm(jaw_end_l_loss - jaw_end_r_loss, ord='euclidean') ** 2
            # ear_end_loss = tf.norm(ear_end_l_loss - ear_end_r_loss, ord='euclidean') ** 2

            # diff0 = [nose_bridge_loss,r_eye_lid_loss,l_eye_lid_loss,lip_loss,r_lip_bend_loss,l_lip_bend_loss,r_lip_jaw_loss,l_lip_jaw_loss,lip_chin_loss,orbital_lower_loss,oribtal_upper_loss,puffer_loss,mouth_corner_loss,jaw_end_loss,ear_end]